# 라이브러리

In [236]:
options(warn =-1, repr.plot.width=15, repr.plot.height=9)
# ## 패키지 설치
library(dplyr)
library(stringr)
library(ggplot2)
library(gridExtra)
library(scales)
library(randomForest)
library(lubridate)
library(caret)
library(e1071)
library(nnet)
library(class)


# 전처리

In [237]:
### .csv 불러오기
data <- read.csv("./data/refined_data_filtered_major5_subset.csv")

In [238]:
data$근무지역분류 <- as.character(data$근무지역분류)
data$근무지역분류 <- factor(data$근무지역분류)
data$장애유형 <- as.character(data$장애유형)
data$장애유형 <- factor(data$장애유형)
data$중증여부 <- as.character(data$중증여부)
data$중증여부 <- factor(data$중증여부)
data$근무지역 <- as.character(data$근무지역)
data$근무지역 <- factor(data$근무지역)
data$취업직종대분류 <- as.character(data$취업직종대분류)
data$취업직종대분류 <- factor(data$취업직종대분류)
data$연령대 <- as.character(data$연령대)
data$연령대 <- factor(data$연령대)
data$장애유형_중증여부 <- as.character(data$장애유형_중증여부)
data$장애유형_중증여부 <- factor(data$장애유형_중증여부)
data$월 <- as.character(data$월)
data$월 <- factor(data$월)

In [239]:
data$target<-(ifelse(data$취업직종대분류=="경영·행정·사무직","경영·행정·사무직",data$취업직종대분류))

data$target <- ifelse(data$취업직종대분류 %in% c("청소 및 기타 개인서비스직", "음식 서비스직", "미용·예식 서비스직", "영업·판매직", "스포츠·레크리에이션직","경호·경비직"), "서비스직", data$target)

data$target <- ifelse(grepl("^\\d+$", data$target), "기타", data$target)

In [240]:
str(data)

'data.frame':	24979 obs. of  11 variables:
 $ 취업일자         : chr  "2020-01-01" "2020-01-01" "2020-01-01" "2020-01-01" ...
 $ 연령             : int  65 61 53 64 53 60 63 40 56 62 ...
 $ 장애유형         : Factor w/ 5 levels "시각장애","정신장애",..: 1 1 1 1 1 1 1 1 4 1 ...
 $ 중증여부         : Factor w/ 2 levels "경증","중증": 2 2 1 2 1 2 2 2 2 2 ...
 $ 근무지역         : Factor w/ 270 levels "강원 강릉시",..: 97 99 114 106 101 111 108 119 18 17 ...
 $ 취업직종대분류   : Factor w/ 21 levels "건설·채굴직",..: 2 2 2 2 2 2 2 9 9 9 ...
 $ 연령대           : Factor w/ 6 levels "1","2","3","4",..: 5 5 4 5 4 5 5 3 4 5 ...
 $ 장애유형_중증여부: Factor w/ 10 levels "시각장애-경증",..: 2 2 1 2 1 2 2 2 8 2 ...
 $ 월               : Factor w/ 12 levels "1","10","11",..: 1 1 1 1 1 1 5 6 10 1 ...
 $ 근무지역분류     : Factor w/ 17 levels "강원","경기",..: 4 4 4 4 4 4 4 4 1 1 ...
 $ target           : chr  "경영·행정·사무직" "경영·행정·사무직" "경영·행정·사무직" "경영·행정·사무직" ...


In [241]:
# 불필요한 컬럼 제거

data_filtered <- subset(data, select = -c(취업일자,연령,장애유형,중증여부,근무지역))
sum(is.na(data_filtered))

[1] 0

In [242]:
# 연령대
data_filtered$age_20 <- data_filtered$연령대 == 1
data_filtered$age_30 <- data_filtered$연령대 == 2
data_filtered$age_40 <- data_filtered$연령대 == 3
data_filtered$age_50 <- data_filtered$연령대 == 4
data_filtered$age_60 <- data_filtered$연령대 == 5
data_filtered$age_70 <- data_filtered$연령대 == 6

In [243]:
# 장애유형_중증여부 바꾸기
data_filtered$시각장애_중증 <- data_filtered$장애유형_중증여부 == "시각장애-중증"
data_filtered$시각장애_경증 <- data_filtered$장애유형_중증여부 == "시각장애-경증"

data_filtered$지체장애_중증 <- data_filtered$장애유형_중증여부 == "지체장애-중증"
data_filtered$지체장애_경증 <- data_filtered$장애유형_중증여부 == "지체장애-경증"

data_filtered$지적장애_경증 <- data_filtered$장애유형_중증여부 == "지적장애-경증"
data_filtered$지적장애_중증 <- data_filtered$장애유형_중증여부 == "지적장애-중증"

data_filtered$청각장애_경증 <- data_filtered$장애유형_중증여부 == "청각장애-경증"
data_filtered$청각장애_중증 <- data_filtered$장애유형_중증여부 == "청각장애-중증"

data_filtered$정신장애_경증 <- data_filtered$장애유형_중증여부 == "정신장애-경증"
data_filtered$정신장애_중증 <- data_filtered$장애유형_중증여부 == "정신장애-중증"

In [244]:
# 근무지역 top 6 남기기 서울 경기 경남 부산 인천 경북 
data_filtered <- data_filtered %>%
  filter(근무지역분류 %in% c("서울","경기","인천"))

In [245]:
#   근무지역 분류 바꾸기

data_filtered$서울 <- data_filtered$근무지역분류 == "서울"
data_filtered$경기 <- data_filtered$근무지역분류 == "경기"
data_filtered$인천 <- data_filtered$근무지역분류 == "인천"


In [246]:
# 월 열을 숫자형으로 변환
data_filtered <- data_filtered %>%
  mutate(월 = as.numeric(월))

# 상반기, 하반기 구분하는 열 추가
data_filtered <- data_filtered %>%
  mutate(상반기 = 월 %in% c(1, 2, 3, 4, 5, 6))

In [247]:
# 월
# data_filtered$Jan <- data_filtered$월  == 01
# data_filtered$Feb <- data_filtered$월  == 02
# data_filtered$Mar <- data_filtered$월  == 03
# data_filtered$Apr <- data_filtered$월  == 04
# data_filtered$May <- data_filtered$월  == 05
# data_filtered$Jun <- data_filtered$월  == 06
# data_filtered$Jul <- data_filtered$월  == 07
# data_filtered$Aug <- data_filtered$월  == 08
# data_filtered$Sep <- data_filtered$월  == 09
# data_filtered$Oct <- data_filtered$월  == 10
# data_filtered$Nov <- data_filtered$월  == 11
# data_filtered$Dec <- data_filtered$월  == 12

In [248]:
str(data_filtered)
sum(is.na(data_filtered))

'data.frame':	11370 obs. of  26 variables:
 $ 취업직종대분류   : Factor w/ 21 levels "건설·채굴직",..: 21 21 21 21 2 2 9 9 9 9 ...
 $ 연령대           : Factor w/ 6 levels "1","2","3","4",..: 1 1 2 1 4 4 1 1 2 1 ...
 $ 장애유형_중증여부: Factor w/ 10 levels "시각장애-경증",..: 6 6 6 6 8 7 6 6 6 4 ...
 $ 월               : num  3 4 3 4 2 2 6 6 6 6 ...
 $ 근무지역분류     : Factor w/ 17 levels "강원","경기",..: 2 2 2 2 2 2 2 2 2 2 ...
 $ target           : chr  "서비스직" "서비스직" "서비스직" "서비스직" ...
 $ age_20           : logi  TRUE TRUE FALSE TRUE FALSE FALSE ...
 $ age_30           : logi  FALSE FALSE TRUE FALSE FALSE FALSE ...
 $ age_40           : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_50           : logi  FALSE FALSE FALSE FALSE TRUE TRUE ...
 $ age_60           : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_70           : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 시각장애_중증    : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 시각장애_경증    : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지체장애_중증    : lo

[1] 0

In [249]:
str(data_filtered)
sum(is.na(data_filtered))

'data.frame':	11370 obs. of  26 variables:
 $ 취업직종대분류   : Factor w/ 21 levels "건설·채굴직",..: 21 21 21 21 2 2 9 9 9 9 ...
 $ 연령대           : Factor w/ 6 levels "1","2","3","4",..: 1 1 2 1 4 4 1 1 2 1 ...
 $ 장애유형_중증여부: Factor w/ 10 levels "시각장애-경증",..: 6 6 6 6 8 7 6 6 6 4 ...
 $ 월               : num  3 4 3 4 2 2 6 6 6 6 ...
 $ 근무지역분류     : Factor w/ 17 levels "강원","경기",..: 2 2 2 2 2 2 2 2 2 2 ...
 $ target           : chr  "서비스직" "서비스직" "서비스직" "서비스직" ...
 $ age_20           : logi  TRUE TRUE FALSE TRUE FALSE FALSE ...
 $ age_30           : logi  FALSE FALSE TRUE FALSE FALSE FALSE ...
 $ age_40           : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_50           : logi  FALSE FALSE FALSE FALSE TRUE TRUE ...
 $ age_60           : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_70           : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 시각장애_중증    : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 시각장애_경증    : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지체장애_중증    : lo

[1] 0

In [250]:
# 불필요한 컬럼 제거
data_filtered_subset <- subset(data_filtered, select = -c(연령대,장애유형_중증여부,월,근무지역분류))

In [251]:
str(data_filtered_subset)
sum(is.na(data_filtered_subset))

'data.frame':	11370 obs. of  22 variables:
 $ 취업직종대분류: Factor w/ 21 levels "건설·채굴직",..: 21 21 21 21 2 2 9 9 9 9 ...
 $ target        : chr  "서비스직" "서비스직" "서비스직" "서비스직" ...
 $ age_20        : logi  TRUE TRUE FALSE TRUE FALSE FALSE ...
 $ age_30        : logi  FALSE FALSE TRUE FALSE FALSE FALSE ...
 $ age_40        : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_50        : logi  FALSE FALSE FALSE FALSE TRUE TRUE ...
 $ age_60        : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_70        : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 시각장애_중증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 시각장애_경증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지체장애_중증 : logi  FALSE FALSE FALSE FALSE TRUE FALSE ...
 $ 지체장애_경증 : logi  FALSE FALSE FALSE FALSE FALSE TRUE ...
 $ 지적장애_경증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지적장애_중증 : logi  TRUE TRUE TRUE TRUE FALSE FALSE ...
 $ 청각장애_경증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 청각장애_중증 : logi  FALSE FALSE FALSE 

[1] 0

In [252]:
# 불필요한 컬럼 제거
data_filtered_subset <- subset(data_filtered, select = -c(연령대,장애유형_중증여부,월,근무지역분류))

In [253]:
str(data_filtered_subset)

'data.frame':	11370 obs. of  22 variables:
 $ 취업직종대분류: Factor w/ 21 levels "건설·채굴직",..: 21 21 21 21 2 2 9 9 9 9 ...
 $ target        : chr  "서비스직" "서비스직" "서비스직" "서비스직" ...
 $ age_20        : logi  TRUE TRUE FALSE TRUE FALSE FALSE ...
 $ age_30        : logi  FALSE FALSE TRUE FALSE FALSE FALSE ...
 $ age_40        : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_50        : logi  FALSE FALSE FALSE FALSE TRUE TRUE ...
 $ age_60        : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_70        : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 시각장애_중증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 시각장애_경증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지체장애_중증 : logi  FALSE FALSE FALSE FALSE TRUE FALSE ...
 $ 지체장애_경증 : logi  FALSE FALSE FALSE FALSE FALSE TRUE ...
 $ 지적장애_경증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지적장애_중증 : logi  TRUE TRUE TRUE TRUE FALSE FALSE ...
 $ 청각장애_경증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 청각장애_중증 : logi  FALSE FALSE FALSE 

# 변동사항

In [254]:
data_filtered_subset$target <- factor(data_filtered_subset$target)

In [255]:
data_filtered_subset <- subset(data_filtered_subset, select = -c(취업직종대분류))

In [256]:
str(data_filtered_subset)
summary(data_filtered_subset)

'data.frame':	11370 obs. of  21 variables:
 $ target       : Factor w/ 3 levels "경영·행정·사무직",..: 3 3 3 3 1 1 2 2 2 2 ...
 $ age_20       : logi  TRUE TRUE FALSE TRUE FALSE FALSE ...
 $ age_30       : logi  FALSE FALSE TRUE FALSE FALSE FALSE ...
 $ age_40       : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_50       : logi  FALSE FALSE FALSE FALSE TRUE TRUE ...
 $ age_60       : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_70       : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 시각장애_중증: logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 시각장애_경증: logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지체장애_중증: logi  FALSE FALSE FALSE FALSE TRUE FALSE ...
 $ 지체장애_경증: logi  FALSE FALSE FALSE FALSE FALSE TRUE ...
 $ 지적장애_경증: logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지적장애_중증: logi  TRUE TRUE TRUE TRUE FALSE FALSE ...
 $ 청각장애_경증: logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 청각장애_중증: logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 정신장애_경증: logi  FALSE FALSE FALSE FALSE FA

              target       age_20          age_30          age_40       
 경영·행정·사무직:4003   Mode :logical   Mode :logical   Mode :logical  
 기타            :3009   FALSE:7633      FALSE:9378      FALSE:9717     
 서비스직        :4358   TRUE :3737      TRUE :1992      TRUE :1653     
   age_50          age_60          age_70        시각장애_중증  
 Mode :logical   Mode :logical   Mode :logical   Mode :logical  
 FALSE:9625      FALSE:9688      FALSE:10809     FALSE:10512    
 TRUE :1745      TRUE :1682      TRUE :561       TRUE :858      
 시각장애_경증   지체장애_중증   지체장애_경증   지적장애_경증  
 Mode :logical   Mode :logical   Mode :logical   Mode :logical  
 FALSE:10664     FALSE:10121     FALSE:8804      FALSE:11362    
 TRUE :706       TRUE :1249      TRUE :2566      TRUE :8        
 지적장애_중증   청각장애_경증   청각장애_중증   정신장애_경증  
 Mode :logical   Mode :logical   Mode :logical   Mode :logical  
 FALSE:7478      FALSE:10851     FALSE:10549     FALSE:11366    
 TRUE :3892      TRUE :519       TRUE :821       TRUE :4    

In [257]:
lm(target~.,data_filtered_subset)


Call:
lm(formula = target ~ ., data = data_filtered_subset)

Coefficients:
      (Intercept)         age_20TRUE         age_30TRUE         age_40TRUE  
         2.814348          -0.932518          -0.901066          -0.770818  
       age_50TRUE         age_60TRUE         age_70TRUE  시각장애_중증TRUE  
        -0.505193          -0.197232                 NA          -0.207169  
시각장애_경증TRUE  지체장애_중증TRUE  지체장애_경증TRUE  지적장애_경증TRUE  
        -0.260786          -0.317394          -0.297051           0.075218  
지적장애_중증TRUE  청각장애_경증TRUE  청각장애_중증TRUE  정신장애_경증TRUE  
         0.380867          -0.067942          -0.024983          -0.409262  
정신장애_중증TRUE           서울TRUE           경기TRUE           인천TRUE  
               NA          -0.038439          -0.004819                 NA  
       상반기TRUE  
        -0.127618  


# 시험준비

In [258]:
samp <- createDataPartition(data_filtered_subset$target, p=0.7, list=F)

#   학습용
data.tr <- data_filtered_subset[samp,]

#   검증용
data.test <- data_filtered_subset[-samp,]

In [259]:
#   KNN을 위한 Training Data 준비
data.training <- subset(data.tr, select = -target)
data.training.label <- data.tr$target

In [260]:
#   문제 및 정답 만들기
x   <- subset(data.test, select = -target)
y   <- data.test$target


# 학습시키기

In [261]:
#  랜덤포레스트 학습 시키기
rf <- randomForest(target ~., data.tr, ntree=500)

In [262]:
# SVM 학습시키기
modelsvm <- svm(target~.,data.tr)

In [263]:
# KNN 학습시키기 3,5,7,
knnmodel <- knn(train=data.training, test=x,cl=data.training.label,k=3)

In [264]:
# nnet 학습시키기
modelnnet <- nnet(target~.,data.tr,size =20)

# weights:  483
initial  value 21115.333766 
iter  10 value 7608.860147
iter  20 value 7374.750784
iter  30 value 7301.545089
iter  40 value 7258.034392
iter  50 value 7225.675809
iter  60 value 7199.208730
iter  70 value 7177.873145
iter  80 value 7156.416244
iter  90 value 7137.859075
iter 100 value 7126.295168
final  value 7126.295168 
stopped after 100 iterations


# 검증하기

In [265]:
#  Random Forest 검증하기
predrf <- predict(rf, x)

In [266]:
#  SVM 검증하기
predmodelsvm <- predict(modelsvm, x)

In [267]:
# KNN 검증하기
mean(knnmodel==y) 

[1] 0.5432678

In [268]:
# nnet 검증하기
prednnet = predict(modelnnet, x,type = "class")    # class : 객관식, raw : 주관식

# 예측력 확인

In [269]:
#  RF 예측력
mean(predrf == y)

[1] 0.5532414

In [270]:
#  SVM 예측력
mean(predmodelsvm == y)

[1] 0.5517747

In [271]:
#  KNN 예측력
mean(knnmodel == y)

[1] 0.5432678

In [272]:
#  nnet 예측력
mean(prednnet == y)

[1] 0.5376943

# 오분류표

In [273]:
#  Random Forest 오분류표
table(predrf,y)

                  y
predrf             경영·행정·사무직 기타 서비스직
  경영·행정·사무직              625  198      265
  기타                          155  392      173
  서비스직                      420  312      869

In [274]:
#  SVM 오분류표
table(predmodelsvm,y)

                  y
predmodelsvm       경영·행정·사무직 기타 서비스직
  경영·행정·사무직              597  184      251
  기타                          146  341      113
  서비스직                      457  377      943

In [275]:
#  KNN 오분류표
# table(y, knnmodel)

In [276]:
# nnet 오분류표
table(prednnet,y)

                  y
prednnet           경영·행정·사무직 기타 서비스직
  경영·행정·사무직              633  203      312
  기타                          156  414      209
  서비스직                      411  285      786

In [277]:
# 새로운 데이터에 대한 예측
predict(rf, (list(age_20=FALSE,age_30=FALSE,age_40=FALSE,age_50=FALSE,age_60=TRUE,age_70=FALSE,
            시각장애_중증=TRUE,시각장애_경증=FALSE,지체장애_경증=FALSE,지체장애_중증=FALSE,지적장애_경증=FALSE,지적장애_중증=FALSE,
            청각장애_경증=FALSE,청각장애_중증=FALSE,정신장애_경증=FALSE,정신장애_중증=FALSE,
            경북=TRUE,강원=FALSE,전남=FALSE,충북=FALSE,경기=FALSE,인천=FALSE,서울=FALSE,울산=FALSE,대전=FALSE,부산=FALSE,
            전북=FALSE,광주=FALSE,충남=FALSE,대구=FALSE,경남=FALSE,제주=FALSE,세종=FALSE,경북=FALSE,
            Jan=TRUE,Feb=FALSE,Mar=FALSE,Apr=FALSE,May=FALSE,Jun=FALSE,Jul=FALSE,Aug=FALSE,Sep=FALSE,
            Oct=FALSE,Nov=FALSE,Dec=FALSE)), type = "prob")

# pred_prob <- predict(rf, new_data, type = "prob")
# pred_prob <- predict(rf, new_data, type = "prob")

ERROR: Error in eval(predvars, data, env): 객체 '상반기'를 찾을 수 없습니다
